In [45]:
import eikon as ek
from tqdm import tqdm
import pandas as pd
from pydash.arrays import chunk

In [46]:
# Login to Eikon API
ek.set_app_key('') # insert api key here

In [47]:
def get_RIC(df):
    results = []  # To store individual DataFrames
    df_cik_list = df['cik'].tolist() # Make a list of all cik
    chunklist = chunk(df_cik_list, 100)  # Split list into chunks of 100
    
    # Add a progress bar using tqdm
    for i in tqdm(range(len(chunklist)), desc="Processing batches"):
            
        try:
            # Call get_data for the current batch
            RIC_dataframe = ek.get_data(instruments= chunklist[i], fields= ['TR.RIC'])         
                
            results.append(RIC_dataframe[0])  # Append DataFrame to results
                
                
        except Exception as e:
            print(f"Error processing batch {i}: {e}")
        
    Result = pd.concat(results, ignore_index=True)
    
    return Result


In [48]:
def save_and_analyze_ric(df, year):

    # First, record the initial number of rows.
    initial_count = len(df)
    
    # Remove rows where 'RIC' is either NaN or an empty string.
    # 1. Drop rows with NaN in the 'RIC' column.
    df = df.dropna(subset=['RIC'])

    # 2. Drop rows where 'RIC' is an empty string (after stripping any whitespace).
    df = df[df['RIC'].str.strip() != '']
    
    # Calculate and print the number of rows lost.
    rows_lost = initial_count - len(df)
    
    with open(f'Data/{year}/{year}_ric_analysis.txt', "a") as f:
        f.write(f"{year} originally has {initial_count} ric values.")
        f.write("\n")
        f.write(f'We lost {rows_lost} rows due to filtering Na')
        f.write("\n")
        f.write(f'Analysis of Na:\n{df.isna().sum()}')
        f.write(f"\n")
        f.write(f'Number of duplicates: \n{df['RIC'].duplicated().sum()}')
        f.write(f"\n")
        
    df['RIC_clean'] = df['RIC'].str.split('^').str[0]
    
    # Now for the extraction of all NASDAQ, NYSE and AMEX endings
    # NYSE = .A; .C; .K; .MW; .P
    # AMEX = .K
    # NASDAQ = .CXC; .CXX; .DF; .DI; .NB; .NBC; .O; .OQ; .PH
    # No BATS and no .UP
    # https://community.developers.refinitiv.com/discussion/92114/how-to-find-the-list-with-the-exchange-for-which-the-suffix-of-a-ric-stands-for/p1?tab=accepted
    # List from 2022
    
    allowed_suffixes =  ['.A', '.C', '.K', '.MW', '.P', '.CXC', '.CXX', 
                      '.DF', '.DI', '.NB', '.NBC', '.O', '.OQ', '.PH']
    
    df = df.loc[df['RIC_clean'].str.endswith(tuple(allowed_suffixes))]

    with open(f'Data/{year}/{year}_ric_analysis.txt', "a") as f:
        f.write(f"We lost {initial_count - len(df)} ric values due to filtering for NASDAQ, NYSE and AMEX")
        f.write("\n")
        f.write(f'We are left with {len(df)}.')
        
        
    df = df.rename(columns={'Instrument': 'cik'})
        
    # Save the cleaned DataFrame
    df.to_parquet(f'Data/{year}/ric_{year}.parquet', engine='fastparquet')
    
    print(f"RIC for {year} has been analyzed and saved into a parquet file.")
        


In [49]:
year = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
for period in year:
    df = pd.read_parquet(f"Data/{period}/cik_{period}.parquet")
    df["cik"] = df["cik"].astype(str)  # Convert cik to string
    
    ric = get_RIC(df)
    ric
    
    save_and_analyze_ric(ric, period)

Processing batches:   0%|          | 0/74 [00:00<?, ?it/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
Processing batches: 100%|██████████| 74/74 [02:51<00:00,  2.32s/it]


RIC for 2016 has been analyzed and saved into a parquet file.


Processing batches:   0%|          | 0/71 [00:00<?, ?it/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
Processing batches: 100%|██████████| 71/71 [02:56<00:00,  2.49s/it]


RIC for 2017 has been analyzed and saved into a parquet file.


Processing batches:   0%|          | 0/70 [00:00<?, ?it/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
Processing batches: 100%|██████████| 70/70 [02:52<00:00,  2.47s/it]


...

RIC for 2021 has been analyzed and saved into a parquet file.


Processing batches:   0%|          | 0/77 [00:00<?, ?it/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
Processing batches: 100%|██████████| 77/77 [03:11<00:00,  2.49s/it]


RIC for 2022 has been analyzed and saved into a parquet file.


Processing batches:   0%|          | 0/73 [00:00<?, ?it/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
Processing batches: 100%|██████████| 73/73 [03:04<00:00,  2.52s/it]


RIC for 2023 has been analyzed and saved into a parquet file.
